## Before we start, let's load a few libraries:

In [ ]:
#module includes
from ipywidgets import interact, interactive_output, interact_manual, interactive, FloatSlider, IntSlider, Dropdown
from IPython.display import display
from math import sin, cos, exp, log, sqrt
from ngsolve import *
from mesh1d import *
from draw1d import * 
from specials import *

# Introduction to the finite element method for the 1D heat conduction problem

- The differential equation:
$$
  - k \frac{d^2 u}{d x^2}(x) = q_Q(x) \quad \text{for all} \quad x \quad \text{ in the intervall } (a,b).
$$

- The boundary conditions:
  - Specified temperature at the edge, e.g: 
$$
  u(a) = T_a \in \mathbb{R}, \qquad u(b) = T_b \in \mathbb{R}. 
$$
  - Specified heat flow at the edge, e.g: 
$$
  k \frac{ d u}{d x}(a) = q_S^a \in \mathbb{R}, \qquad - k \frac{ d u}{d x}(b) = q_S^b \in \mathbb{R}.
$$

## Unknown quantity is a function

- The solution of the differential equation is a function 
- A function has “infinitely many degrees of freedom”, example:
    - Let $f(x)$ be a function in the interval [0,1]
    - We fix $N+1$ degrees of freedom at $N+1$ points: $f(x) = 0$ for $x = \frac{0}{N}, \frac{1}{N}, \frac{2}{N}, ... , \frac{N-1}{N}, \frac{N}{N}$
    - Nevertheless, there are functions $f(x)$ that are not 0 everywhere:

In [ ]:
interact(DrawSpecialFunction,N=(1,193,12));

- The considered functions are no longer “smooth” for $N \to \infty$
- If the function we are looking for is “smooth”, the function can be approximated well with a finite number of degrees of freedom:

In [ ]:
align(interactive(DrawErrorToSmoothFunction,N=(10,100,10),smooth_function=["sin(3*pi*x)*cos(5*pi*x)",
                     "x**8+6*x**4","sin(8*pi*x)"]))

## Basic idea discretization of partial differential equations

 1. choose representation of the function so that the function is still well approximated 
   
   (large $N$ is better (more accurate))
   
 2. determine equations for the $N$ unknowns
 
 3. solve equations 
   
   (small $N$ is better (more economical))

## 1. “Approximation” of a “smooth” function

### Degrees of freedom for approximating a function:
   $
       u_h(x) = c_0 \varphi_0(x) + c_1 \varphi_1(x) + ... + c_{n-1} \varphi_{n-1}(x) = \sum_{i=0}^{n-1} c_i \varphi_i(x)
   $
   - The coefficients $c_i$ are the unknowns (“degrees of freedom”), $i=0,...,n-1$.
   - $\varphi_i(x)$ are the “basis functions”.

In [ ]:
align(interactive(DrawBasisFunction, N=[4,6,8], i=IntSlider(min=0, max=41, step=1, continuous_update=False),
                  order=[1,2,3]))

### The space of all functions of the form $u_h(x) = \sum_{i=0}^{n-1} c_i \varphi_i(x)$
 - All functions that can be represented as linear combinations of the basic functions are collected in the **"Finite element space"**:
 $$
   V_h \text{ is the set of all functions }u_h \text{ with } u_h(x) = \sum_{i=0}^{n-1} c_i \varphi_i(x)
 $$

In [ ]:
align(interactive(ApproximateWithFESpace, 
                  N=[4,8,16,32,64], 
                  order=[1,2,3], 
                  f=["sin(3·pi·x)",
                     "sin(3·pi·x)·cos(5·pi·x)"]))

### Visualization of the "partial sum" $u_h^j = \sum_{i=0}^{\color{red}j} c_i \varphi_i(x)$:

In [ ]:
align(interactive(PartialApproximateWithFESpace, N=[4,8,16,32,64], 
                  order=[1,2,3], f=["sin(3·pi·x)","sin(3·pi·x)·cos(5·pi·x)"], j=IntSlider(min=-1, max=200, step=1, continuous_update=False)))

## 2. Determination of the equations for the unknowns

#### Goal:

* Find $n$ equations for the $n$ unknowns: 
* Since the differential equation is linear, we expect to be able to find $n$ linear equations. 
* So we first look for a linear system of equations of the form 
 $$
 \left( \begin{array}{cccc} 
 A_{0,0} & A_{0,1} & \dots & A_{0,n-1} \\ 
 A_{1,0} & A_{1,1} & \dots & A_{1,n-1} \\ 
 \vdots & \vdots & \ddots & \vdots \\ 
 A_{n-1,0} & A_{n-1,1} & \dots & A_{n-1,n-1} \\ 
 \end{array}
 \right) \cdot 
  \left( \begin{array}{c} 
 c_{0} \\ c_1 \\ \vdots \\ c_{n-1}
 \end{array}
 \right)
 =
  \left( \begin{array}{c} 
 f_{0} \\ f_1 \\ \vdots \\ f_{n-1}
 \end{array}
 \right)
  $$
  to determine the coefficients.
  
* Attention: Numbering from $0$ to $n-1$ (alternatively also possible from $1$ to $n$).

- Require $n$ equations for $n$ unknowns
- In the finite element method, we require the partial differential equation **not at points**
- Instead, we first reformulate the partial differential equation:


\begin{align*}
    - k \frac{d^2 u}{d x^2}(x) &= q_Q(x) \text{ in } (a,b), \\    \text{multiply by (almost) any} & \text{ function } v(x) \text{ and integrate over the domain: } \\
    \Longrightarrow \int_a^b - k \frac{d^2 u}{d x^2}(x) v(x) \ dx &= \int_a^b q_Q(x) v(x) \ dx
\end{align*}

* $\leadsto$ [Exercise on partial integration and boundary conditions](../../../Uebungen/blatt_zu_1DFEM.pdf)


$$
    \Longrightarrow  \int_a^b k \frac{d u}{d x}(x) \frac{d v}{d x}(x) \ dx - \Big[k \frac{d u}{dx} v(x) \Big]_a^b = \int_a^b q_Q(x) v(x) \ dx
    $$
   

* For the sake of simplicity, we assume that $u(a)$ and $u(b)$ are given (Dirichlet boundary conditions), i.e. 
$u(a)=T^a$, $u(b)=T^b$
* The “freedom” that we take from the unknown function $u$, we also take from the “test function” $v$ and set $v(a) = v(b) = 0$
* This simplifies the problem to a (weak) formulation (FEM formulation):
Find the function $u(x)$ with $u(a)=T^a$, $u(b)=T^b$, so that for (almost) every function $v(x)$ with $v(a)=v(b)=0$ holds
$$ \int_a^b k \frac{d u}{d x}(x) \frac{d v}{d x}(x) \ dx = \int_a^b q_Q(x) v(x) \ dx. $$
* Corresponding modified left and right side for other boundary conditions (see exercise)

- Discretization by requirement:

  Find a function $u_h(x)$ with $u_h = T^a \cdot \varphi_0 (x) + \sum_{\color{red}{j=1}}^{\color{red}{n-2}} c_j \varphi_j(x) + T^b \cdot \varphi_{n-1} (x)$ such that 
  $$ \int_a^b k \frac{d u_h}{d x}(x) \frac{d v_h}{d x}(x) \ dx = \int_a^b q_Q(x) v_h(x) \ dx $$
  for all functions $v_h \in V_{h,0} = V_h \cap \{v(a)=v(b)=0\}$, i.e. all functions in $V_h$ that also fulfill the boundary condition $v(a)=v(b)=0$.  

- Equivalently, we can ask:

  Find the coefficients $c_j$, $j = 1,...,n-2$, such that
  $$ \int_a^b k \frac{d u_h}{d x}(x) \frac{d \varphi_i}{d x}(x) \ dx = \int_a^b q_Q(x) \varphi_i(x) \ dx $$
  for all $i = 1,...,n-2$.
 
 - I.e. we have $n-2$ equations for $n-2$ unknowns.

 * with $u_h = T^a \cdot \varphi_0 (x) + \sum_{\color{red}{j=1}}^{\color{red}{n-2}} c_j \varphi_j(x) + T^b \cdot \varphi_{n-1} (x)$ we can also write:
 
 Find the coefficients $c_j$, $j = 1,..,n-2$, so that for all $i = 1,..,n-2$ 
  $$ \sum_{{i=1}}^{{n-2}} c_j \underbrace{\int_a^b k \frac{d \varphi_j}{d x}(x) \frac{d \varphi_i}{d x}(x) \ dx}_{A_{i,j}} = f_i \qquad \text{with}$$
  $$
  f_i = \int_a^b q_Q(x) \varphi_i(x) \ dx - T^a \int_a^b k \frac{d \varphi_0}{d x}(x) \frac{d \varphi_i}{d x}(x) \ dx - T^b \int_a^b k \frac{d \varphi_{n-1}}{d x}(x) \frac{d \varphi_i}{d x}(x) \ dx
  $$
  

 

$\leadsto$ Exercise: Calculate the step from the last slides to this one.

 * This corresponds to a matrix-vector equation
 $$
 \left( \begin{array}{cccc} 
 A_{1,1} & A_{1,2} & \dots & A_{1,n-2} \\ 
 A_{2,1} & A_{2,2} & \dots & A_{2,n-2} \\ 
 \vdots & \vdots & \ddots & \vdots \\ 
 A_{n-2,1} & A_{n-2,2} & \dots & A_{n-2,n-2} \\ 
 \end{array}
 \right) \cdot 
  \left( \begin{array}{c} 
 c_{1} \\ c_2 \\ \vdots \\ c_{n-2}
 \end{array}
 \right)
 =
  \left( \begin{array}{c} 
 f_{1} \\ f_2 \\ \vdots \\ f_{n-2}
 \end{array}
 \right)
  $$
  
 * Solving the linear equations with specialized solvers, e.g. based on Gaussian elimination 
 * How are the matrix entries composed?

#### Calculation of the matrix entries

In [ ]:
ComputeMatrixEntry(N=12,order=1,k=1)

#### “Sparse” system matrix: $A_{i,j} = 0$ for most $(i,j)$ pairs

In [ ]:
align(interactive(Spy,N=[1,2,3,4,8,16],order=[1,2,3,4]))

#### Exercise:

* Consider $N = 3$, `order`$=1$ and $k=1$ and $q_Q=1$
* There are $n=4$ coefficients to be determined
* We also require that the first and last coefficients are 0 (boundary condition)
* Determine the matrix entries $A_{1,1}$, $A_{1,2}$, $A_{2,1}$, $A_{2,2}$ 
* Solve the system of equations
\begin{align*}
\begin{array}{rl}
  A_{1,1} c_1 + A_{1,2} c_2 &= f_1 \\
  A_{2,1} c_1 + A_{2,2} c_2 &= f_2 
\end{array} 
\qquad \Longleftrightarrow \qquad
\left( 
\begin{array}{rl}
  A_{1,1} & A_{1,2} \\
  A_{2,1} & A_{2,2} 
\end{array}
\right)
\cdot
\left(
\begin{array}{c} 
c_1 \\ 
c_2 
\end{array}
\right)=
\left(
\begin{array}{c} f_1 \\ f_2 \end{array} 
\right)
\end{align*}
 for the determined coefficients $A_{1,1}$, $A_{1,2}$, $A_{2,1}$, $A_{2,2}$ and
 $$
 f_1 = \int_{\Omega} \underbrace{q_Q}_{=1} \varphi_1 dx = h = 1/N = 1/3, \qquad
 f_2 = \int_{\Omega} \underbrace{q_Q}_{=1} \varphi_2 dx = h = 1/N = 1/3
 $$
* Sketch the solution

## 4. Model calculations

 ![alt](heat1d.png) 

#### The differential equation:
$$
  - k(x) \frac{d^2 u}{d x^2}(x) = q_Q(x) \quad \text{for all} \quad x \quad \text{ in the intervall } (a,b),
$$
where 
$$
  k(x) = \left\{ \begin{array}{cl} k_1 & \text{ for } x < 0.5 (a+b), \\ k_2 & \text{ else. }  \end{array} \right. , \qquad
  q_Q(x) = \left\{ \begin{array}{cl} q_Q^1 & \text{ for } x < 0.5(a+b), \\ q_Q^2 & \text{ else. }\end{array} \right.
$$
#### Possible boundary conditions

 \begin{align*}
 \text{"Dirichlet":} & & u(a) &= T^a, &\qquad u(b) &= T^b, \\
 \text{"Neumann":} & &k \frac{d u}{dx}(a) &= q_S^a, &\qquad - k \frac{d u}{dx}(b) &= q_S^b, \\
 \text{"Robin":} & & k \frac{d u}{dx}(a) &= \alpha^a  (T^a - u(a)), & \qquad - k \frac{d u}{dx}(b) &= \alpha^b (T^b - u(b)), \\
 \end{align*}
 

In [ ]:
Heat1DExample()

#### Exercise:
Solve the following problems (numerically) and vary $k_1,~k_2$ and $q_Q^1,~q_Q^2$:
* 1.:
  * Solve the problem from the last task again numerically and compare your result.
* 2.:
  * There should be no heat flow at the right edge
  * A temperature of 20°C is measured at the left edge
  * Heating is applied on the whole interval $[0,1]$
  * What is the temperature at the right edge?
* 3.:
  * Outside $[0,1]$ there is a temperature of $\bar u = 20°C$ and the heat transfer law applies:
    $$
      q_s \sim u - T^{\text{ref}}
    $$
  * Heating takes place over the entire interval
  * The thermal conductivity on the right is 10 times greater than on the left

In [ ]:
Heat1DFEM( N=2, order=2, 
           k1 = 30, k2 = 30, 
           Q1 = 30000, Q2= 30000, 
           boundary_condition_left = "Dirichlet", boundary_condition_right = "Dirichlet", 
           value_left = 20, value_right = 20, 
           q_value_left = 0, q_value_right = 1, 
           r_value_left = 0, r_value_right = 1,
           intervalsize = 0.14)

### Heat losses via the surface area

* If we now include losses via the lateral surface again, we obtain

\begin{align*}
    - k \frac{d^2 u}{d x^2}(x) + \bar{\alpha} (u-\bar{T}) &= q_Q(x) \text{ in } (a,b)
\end{align*}

where $\bar{T}$ is the external temperature (assumed to be constant) and $\bar{\alpha}$ is the heat transfer coefficient for the heat transfer across the lateral surface.


* The problem for the (weak) formulation (FEM formulation) with Dirichlet boundary conditions thus becomes:
Find the function $u(x)$ with $u(a)=T^a$, $u(b)=T^b$, so that for (almost) every function $v(x)$ with $v(a)=v(b)=0$ holds

$$ \int_a^b k \frac{d u}{d x}(x) + \bar{\alpha} u(x) v(x) \frac{d v}{d x}(x) \ dx = \int_a^b (q_Q(x)+\bar{\alpha} \bar{T}) v(x) \ dx. $$


* Corresponding modified left and right sides with other boundary conditions are obtained analogously to the previously considered case.

In [ ]:
  Heat1DExample_with_ref()

In [ ]:
Heat1DFEM( N=10, order=3 , 
           k1 = 50, k2 = 50, 
           Q1 = 1250000, Q2= 1250000, 
           boundary_condition_left = "Dirichlet", boundary_condition_right = "Dirichlet", 
           value_left = 20, value_right = 0, 
           q_value_left = 10, q_value_right = 1, 
           r_value_left = 0, r_value_right = 1,
           value_ref=22, c_ref = 100000,
           intervalsize = 0.14)